# 2 Shock Theory using conservation equations

In [1]:
import numpy as np 
from scipy.optimize import fsolve,fmin,root,approx_fprime

class state(object):
  def __init__(self,p=100000,M=1.5,T=300,R=287,gamma=1.4,theta=10*np.pi/180):
    self.p=p
    self.M=M
    self.T=T
    self.gamma =gamma
    self.theta=theta
    self.R=R
    self.phi=0.1
    self.rho=p/(R*self.T)
    self.u=self.M*np.sqrt(self.gamma*self.R*self.T)
    
  def recalc(self):
    self.M = self.u/np.sqrt(self.gamma*self.R*self.T)
    self.p = self.rho*self.R*self.T
    
    
    
def TwoST(state0,theta):
    state1 = nextState(state0,theta)
    state2 = nextState(state1,theta)
    return state1, state2

def ThreeST(state0,theta1):
    state1 = nextState(state0,theta1)
    
    

def getAngles(s0,s1,theta1):
    x = fsolve(lambda x: angFunc(x,s0.u,s1.u,s0.T,s1.T,s0.rho,s1.rho,theta1),np.array([np.pi/16,np.pi/16,np.pi/8]))
    #x = root(lambda x: angFunc(x,s0.u,s1.u,s0.T,s1.T,s0.rho,s1.rho,theta1),np.array([np.pi/4,np.pi/4,np.pi/4]),method='hybr')
    return x
    
    
def angFunc(alpha,u0,u1,T0,T1,rho0,rho1, theta1,R=287):
    """
    alpha[0] - phi2
    alpha[1] - phi3
    alpha[2] - theta2
    """
    f1 = phiFun(alpha[0],T1,u1,alpha[2])
    f2 = phiFun(alpha[1],T0,u0,theta1 - alpha[2])
    f3 = rho1*R*T1 - rho0*R*T0 +\
    (np.tan(alpha[1])/np.tan(alpha[1]-theta1+alpha[2]))*(np.cos(alpha[1])/np.cos(alpha[1]-theta1+alpha[2]))**2\
    *rho0*u0**2*(np.sin(alpha[1]-theta1+alpha[2]))**2 -\
    (np.tan(alpha[0])/np.tan(alpha[0]-alpha[2]))*(np.cos(alpha[0])/np.cos(alpha[0]-alpha[2]))**2\
    *rho1*u1**2*(np.sin(alpha[0]-alpha[2]))**2 +\
    rho1*u1**2*(np.sin(alpha[0]))**2-rho0*u0**2*(np.sin(alpha[1]))**2
    return np.array([f1,f2,f3])
    
    
    
def nextState(s0,theta,weak='True'):
    s1 = state(M=1.2,p=0,T=350, theta=theta)
    p = phi(s0.T,s0.u,s1.theta)
    if weak =='True':
        s1.phi = p[0]
    else:
        s1.phi = p[1]
    s1.rho = rho1(s0.rho,s1.phi,s1.theta)
    s1.u   = u1(s0.u,s1.phi,s1.theta)
    s1.T   = T1(s0.T,s0.u,s1.phi,s1.theta)
    s1.recalc()
    return s1
    

def rho1(rho0,phi,theta):
    return rho0*np.tan(phi)/np.tan(phi-theta)
def u1(u0,phi,theta):
    return u0*np.cos(phi)/np.cos(phi-theta)
def T1(T0,u0,phi,theta,R=287,gamma=1.4):
    return T0+0.5*u0**2*((np.sin(phi))**2-(np.cos(phi)*np.tan(phi-theta))**2)/(gamma*R/(gamma-1))

def phi(T,u,theta,R=287,gamma=1.4):
    out = fsolve(lambda x: phiFun(x,T,u,theta),0.01)
    out = np.append(out, fsolve(lambda x: phiFun(x,T,u,theta),np.pi/2))
    return out

def theta(T,u,phi,R=287,gamma=1.4):
    out = fsolve(lambda x: phiFun(phi,T,u,x),0.01)
    out = np.append(out, fsolve(lambda x: phiFun(phi,T,u,x),np.pi/2))
    return out

def phiFun(phi,T,u,theta,R=287,gamma=1.4):
    f1 = T+0.5*u**2*((np.sin(phi))**2-(np.cos(phi)*np.tan(phi-theta))**2)/(gamma*R/(gamma-1))
    f2 = (T+u**2*np.sin(phi)*(np.sin(phi)-np.tan(phi-theta)*np.cos(phi))/R)*np.tan(phi-theta)/np.tan(phi)
    return f1-f2

## 2ST using conservation equations

In [2]:
s0  = state(M=2,p=100000, T=300, theta=10*np.pi/180)

In [3]:
s1,s2 = TwoST(s0,10*np.pi/180)

In [4]:
s0.rho

1.1614401858304297

In [5]:
s0.u

694.3774189876857

In [6]:
s1.rho

1.6938741148814198

In [7]:
s1.u

616.1248420205723

In [8]:
s1.T

351.045385288763

In [9]:
s1.phi*180/np.pi 

39.31393184481875

In [10]:
s2.rho

2.4059689908267488

In [11]:
s2.u

518.9326955467594

In [12]:
s2.T

405.9576194587229

In [13]:
s2.phi*180/np.pi

49.38404168280174